In [1]:
import nest_asyncio
import os
import sys
import logging

from dotenv import load_dotenv, find_dotenv

from llama_index.core import (
    SimpleDirectoryReader,
    ServiceContext,
    StorageContext,
    VectorStoreIndex,
)
import logging
import sys
import nest_asyncio

nest_asyncio.apply()
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().handlers = []
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index.core import (
    SimpleDirectoryReader,
    StorageContext,
    VectorStoreIndex,
)
from llama_index.legacy.retrievers.bm25_retriever import BM25Retriever
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.node_parser import SentenceSplitter
from llama_index.llms.openai import OpenAI

from llama_index.core.response.notebook_utils import display_source_node
from llama_index.core.retrievers import RouterRetriever
from llama_index.core.tools import RetrieverTool
from llama_index.core.query_engine import RetrieverQueryEngine
# from llama_index.core.retrievers import BM25Retriever
# from llama_index.llms import OpenAI, Ollama
# from llama_index.embeddings import OllamaEmbedding

from llama_index.core.postprocessor import SentenceTransformerRerank
# from llama_index import QueryBundle
from llama_index.core.schema import QueryBundle, NodeWithScore
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().handlers = []
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

nest_asyncio.apply()

documents = SimpleDirectoryReader("../data/paul_graham").load_data()


[nltk_data] Downloading package stopwords to
[nltk_data]     /opt/anaconda3/envs/llm/lib/python3.10/site-
[nltk_data]     packages/llama_index/legacy/_static/nltk_cache...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     /opt/anaconda3/envs/llm/lib/python3.10/site-
[nltk_data]     packages/llama_index/legacy/_static/nltk_cache...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
class AdvancedRAG:
    def __init__(self):
        _ = load_dotenv(find_dotenv())
        # load documents
        self.documents = SimpleDirectoryReader("../data/", required_exts=['.pdf']).load_data()

        # global variables used later in code after initialization
        self.retriever = None
        self.reranker = None
        self.query_engine = None

        self.bootstrap()

    def bootstrap(self):
        # initialize LLMs
        

        llm = OpenAI(model="qwen",
                    api_key=os.getenv("OPENAI_API_KEY"),
                    temperature=0,
                    system_prompt="You are an expert on the Inflamatory Bowel Diseases and your job is to answer questions. Assume that all questions are related to the Inflammatory Bowel Diseases (IBD). Keep your answers technical and based on facts – do not hallucinate features.")

        # initialize service context (set chunk size)
        service_context = ServiceContext.from_defaults(chunk_size=1024, llm=llm)
        nodes = service_context.node_parser.get_nodes_from_documents(self.documents)

        # initialize storage context (by default it's in-memory)
        storage_context = StorageContext.from_defaults()
        storage_context.docstore.add_documents(nodes)

        index = VectorStoreIndex(
            nodes=nodes,
            storage_context=storage_context,
            service_context=service_context,
        )

        # We can pass in the index, doctore, or list of nodes to create the retriever
        self.retriever = BM25Retriever.from_defaults(similarity_top_k=2, index=index)

        # reranker setup & initialization
        self.reranker = SentenceTransformerRerank(top_n=1, model="/Users/heitao/my_blog/models/models/Xorbits/bge-reranker-base")

        self.query_engine = RetrieverQueryEngine.from_args(
            retriever=self.retriever,
            node_postprocessors=[self.reranker],
            service_context=service_context,
        )

    def query(self, query):
        # will retrieve context from specific companies
        nodes = self.retriever.retrieve(query)
        reranked_nodes = self.reranker.postprocess_nodes(
            nodes,
            query_bundle=QueryBundle(query_str=query)
        )

        print("Initial retrieval: ", len(nodes), " nodes")
        print("Re-ranked retrieval: ", len(reranked_nodes), " nodes")

        for node in nodes:
            print(node)

        for node in reranked_nodes:
            print(node)

        response = self.query_engine.query(str_or_query_bundle=query)
        return response

In [3]:
if __name__ == "__main__":
    adv_rag = AdvancedRAG()
    resp = adv_rag.query("Transformers are ?")
    print(resp)

Previous trailer can not be read ("invalid literal for int() with base 10: b'/Root'",)
Object 71 0 found
Object 70 0 found
Object 3 0 found
Object 2 0 found
Object 15 0 found
Object 23 0 found
Object 29 0 found
Object 34 0 found
Object 39 0 found
Object 45 0 found
Object 50 0 found
Object 60 0 found
Object 59 0 found
Object 67 0 found
Object 5 0 found
Object 7 0 found
Object 8 0 found
Object 186 0 found
Object 185 0 found
Object 187 0 found
Object 9 0 found
Object 191 0 found
Object 190 0 found
Object 192 0 found
Object 10 0 found
Object 4 0 found
Object 11 0 found
Object 17 0 found
Object 22 0 found
Object 198 0 found
Object 197 0 found
Object 16 0 found
Object 18 0 found
Object 19 0 found
Object 20 0 found
Object 25 0 found
Object 28 0 found
Object 201 0 found
Object 200 0 found
Object 24 0 found
Object 26 0 found
Object 31 0 found
Object 30 0 found
Object 32 0 found
Object 33 0 found
Object 36 0 found
Object 37 0 found
Object 203 0 found
Object 35 0 found
Object 38 0 found
Object 41

/var/folders/5b/dn__zk295kn6p0g2sr593nyr0000gn/T/ipykernel_58647/2535406003.py:24: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(chunk_size=1024, llm=llm)


ValueError: Unknown model 'qwen'. Please provide a valid OpenAI model name in: gpt-4, gpt-4-32k, gpt-4-1106-preview, gpt-4-0125-preview, gpt-4-turbo-preview, gpt-4-vision-preview, gpt-4-0613, gpt-4-32k-0613, gpt-4-0314, gpt-4-32k-0314, gpt-3.5-turbo, gpt-3.5-turbo-16k, gpt-3.5-turbo-0125, gpt-3.5-turbo-1106, gpt-3.5-turbo-0613, gpt-3.5-turbo-16k-0613, gpt-3.5-turbo-0301, text-davinci-003, text-davinci-002, gpt-3.5-turbo-instruct, text-ada-001, text-babbage-001, text-curie-001, ada, babbage, curie, davinci, gpt-35-turbo-16k, gpt-35-turbo, gpt-35-turbo-0125, gpt-35-turbo-1106, gpt-35-turbo-0613, gpt-35-turbo-16k-0613

In [4]:
model = "qwen72chat"
url = "http://220.250.27.14:8001/v1"
llm = OpenAI(model="qwen72chat",
            api_key='',
            api_base = url,
            temperature=0,
            system_prompt="You are an expert on the Inflamatory Bowel Diseases and your job is to answer questions. Assume that all questions are related to the Inflammatory Bowel Diseases (IBD). Keep your answers technical and based on facts – do not hallucinate features.")


In [5]:
llm.chat(messages = '111')

ValidationError: 1 validation error for LLMChatStartEvent
messages
  value is not a valid list (type=type_error.list)